# Preprocessing data

In [51]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [14]:
df = pd.read_csv("raw_dataset/tracks.csv")

In [15]:
df

,id,name,popularity,duration_ms,explicit,artists,id_artists,release_date,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
0,35iwgR4jXetI318WEWsa1Q,Carve,6,126903,0,['Uli'],['45tIt06XoI0Iio4LBEVpls'],1922-02-22,0.645,0.4450,0,-13.338,1,0.4510,0.674,0.744000,0.1510,0.1270,104.851,3
1,021ht4sdgPcrDgSk7JTbKY,Capítulo 2.16 - Banquero Anarquista,0,98200,0,['Fernando Pessoa'],['14jtPCOoNZwquk5wd9DxrY'],1922-06-01,0.695,0.2630,0,-22.136,1,0.9570,0.797,0.000000,0.1480,0.6550,102.009,1
2,07A5yehtSnoedViJAZkNnc,Vivo para Quererte - Remasterizado,0,181640,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.434,0.1770,1,-21.180,1,0.0512,0.994,0.021800,0.2120,0.4570,130.418,5
3,08FmqUhxtyLTn6pAh6bk45,El Prisionero - Remasterizado,0,176907,0,['Ignacio Corsini'],['5LiOoJbxVSAMkBS2fUm3X2'],1922-03-21,0.321,0.0946,7,-27.961,1,0.0504,0.995,0.918000,0.1040,0.3970,169.980,3
4,08y9GfoqCWfOGsKdwojr5e,Lady of the Evening,0,163080,0,['Dick Haymes'],['3BiJGZsyX9sJchTqcSA7Su'],1922,0.402,0.1580,3,-16.900,0,0.0390,0.989,0.130000,0.3110,0.1960,103.220,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
586667,5rgu12WBIHQtvej2MdHSH0,云与海,50,258267,0,['阿YueYue'],['1QLBXKM5GCpyQQSVMNZqrZ'],2020-09-26,0.560,0.5180,0,-7.471,0,0.0292,0.785,0.000000,0.0648,0.2110,131.896,4
586668,0NuWgxEp51CutD2pJoF4OM,blind,72,153293,0,['ROLE MODEL'],['1dy5WNgIKQU6ezkpZs4y8z'],2020-10-21,0.765,0.6630,0,-5.223,1,0.0652,0.141,0.000297,0.0924,0.6860,150.091,4
586669,27Y1N4Q4U3EfDU5Ubw8ws2,What They'll Say About Us,70,187601,0,['FINNEAS'],['37M5pPGs6V1fchFJSgCguX'],2020-09-02,0.535,0.3140,7,-12.823,0,0.0408,0.895,0.000150,0.0874,0.0663,145.095,4
586670,45XJsGpFTyzbzeWK8VzR8S,A Day At A Time,58,142003,0,"['Gentle Bones', 'Clara Benin']","['4jGPdu95icCKVF31CcFKbS', '5ebPSE9YI5aLeZ1Z2g...",2021-03-05,0.696,0.6150,10,-6.212,1,0.0345,0.206,0.000003,0.3050,0.4380,90.029,4


In [96]:
new_df = df.drop(columns=["name","popularity","duration_ms","artists","id_artists","release_date"])
new_df["sample_mp3_url"]=""

# Use Spotify API to get 30 seconds sample audio URL

In [36]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json

with open("credentials.json","r") as f:
    credentials = json.loads(f.read())

auth_manager = SpotifyClientCredentials(client_id=credentials["client_id"], client_secret=credentials["client_secret"])
sp = spotipy.Spotify(auth_manager=auth_manager)

result = sp.tracks(tracks=["35iwgR4jXetI318WEWsa1Q","021ht4sdgPcrDgSk7JTbKY"])

for track in result["tracks"]:
    print(track["preview_url"])

https://p.scdn.co/mp3-preview/2f41db376b64c4b148cf426ef9a86e16ebcc92d9?cid=4ffd8b3c6aac481ea6b01a48d4f6d478
https://p.scdn.co/mp3-preview/d9c4623d97c4e872eccefae6bfa97a46094cc481?cid=4ffd8b3c6aac481ea6b01a48d4f6d478


In [97]:
url_list = []
for batch_i in tqdm(range(len(new_df["id"])//50+1)):
    tracks = new_df["id"][batch_i*50:batch_i*50+50]
    result = sp.tracks(tracks=tracks)
    for i in range(len(result["tracks"])):
        new_df.loc[batch_i*50+i,"sample_mp3_url"]=result["tracks"][i]["preview_url"]
    new_df[batch_i*50:batch_i*50+50].to_csv("dataset/tracks_url.csv",mode='a', index=False, header=False)

  0%|          | 7/11734 [00:02<1:04:05,  3.05it/s]


KeyboardInterrupt: 

In [94]:
new_df.iloc[0]

id                                                  35iwgR4jXetI318WEWsa1Q
explicit                                                                 0
danceability                                                         0.645
energy                                                               0.445
key                                                                      0
                                               ...                        
(395, sample_mp3_url)    https://p.scdn.co/mp3-preview/d11d4f5ab4a097d6...
(396, sample_mp3_url)    https://p.scdn.co/mp3-preview/dd35d4e16f595895...
(397, sample_mp3_url)    https://p.scdn.co/mp3-preview/a402ece2feaf5aec...
(398, sample_mp3_url)    https://p.scdn.co/mp3-preview/7f411832ef90b921...
(399, sample_mp3_url)    https://p.scdn.co/mp3-preview/c45608cc50c7f45c...
Name: 0, Length: 415, dtype: object